In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class NaiveBayes(object):
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        
        # mean, variance, priors
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors =  np.zeros(n_classes, dtype=np.float64)

        # extracting mean, variance and priors for each class
        # useful in calculating pdf during prediction
        for c in self._classes:
            X_c = X[y==c]
            self._mean[c, :] = X_c.mean(axis=0)
            self._var[c, :] = X_c.var(axis=0)
            self._priors[c] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditional = np.sum(np.log(self.gaussian_pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)
            
        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]
            

    def gaussian_pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(- (x-mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy
from sklearn.datasets import load_iris

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [3]:
nb = NaiveBayes()
nb.fit(X_train, y_train.ravel())
y_pred = nb.predict(X_test)
print(f"Naive Bayes accuracy: {accuracy(y_test, y_pred)}")

Naive Bayes accuracy: 0.9473684210526315


In [5]:
# Diabetes Dataset
cols = ["Pregnancies" ,"Glucose" ,"BloodPressure" ,"SkinThickness" ,"Insulin" ,"BMI" ,"DiabetesPedigreeFunction" ,"Age" ,"Outcome"]
url = r'diabetes_data.csv'
diabetes_data = pd.read_csv(url, skiprows=9, header=None, names=cols)
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,2,197,70,45,543,30.5,0.158,53,1
1,8,125,96,0,0,0.0,0.232,54,1
2,4,110,92,0,0,37.6,0.191,30,0
3,10,168,74,0,0,38.0,0.537,34,1
4,10,139,80,0,0,27.1,1.441,57,0


In [6]:
diabetes_data.shape

(760, 9)

In [7]:
X = diabetes_data[cols[:-1]].values
y = diabetes_data[cols[-1]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
nb = NaiveBayes()
nb.fit(X_train, y_train.ravel())
y_pred = nb.predict(X_test)
print(f"Naive Bayes accuracy: {accuracy(y_test, y_pred)}")

Naive Bayes accuracy: 0.6973684210526315


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1-Score: {f1_score(y_test, y_pred)}")

Precision: 0.56
Recall: 0.5384615384615384
F1-Score: 0.5490196078431373
